In [19]:
import logging
import os
import sys
import importlib
import argparse
import munch
import yaml
from utils.vis_utils import plot_single_pcd
from utils.train_utils import *
from utils.ri_utils import *
from utils.model_utils import *
from dataset import ShapeNetH5
config_path = 'cfgs/ri_vrcnet.yaml'
args = munch.munchify(yaml.safe_load(open(config_path)))
exp_name = os.path.basename(args.load_model)
log_dir = os.path.dirname(args.load_model)

In [20]:
dataset = ShapeNetH5(train=False, novel_input_only=True, npoints=args.num_points)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size,
                                              shuffle=False, num_workers=int(args.workers))
dataset_length = len(dataset)
print('Length of dataset:', len(dataset))

(10400, 2048, 3)
(400, 2048, 3)
(10400,)
Length of dataset: 10400


In [25]:
data = torch.from_numpy(dataset.gt_data)
input_clone = torch.clone(data)
device = torch.device('cuda:5')
a1, a2, a3, data_new = point_ortho_feature(data)
data_new = inverse_point_ortho_feature(a1, a2, a3, data_new)
data_old = input_clone.to(device)
data_rec = data_new.to(device)
cd_p, cd_t = calc_cd(data_old, data_rec)
print(cd_p.mean())
print(cd_t.mean())

tensor(3.5020e-08, device='cuda:5')
tensor(4.0078e-15, device='cuda:5')


In [22]:
def point_ortho_feature(points, pca=True, axis1=None, axis2=None, axis3=None):
    # Input shape: [batch_size, num_point, 3]
    # Return: [batch_size, num_point, 3]
    batch_size = points.size()[0]
    num_point = points.size()[1]
    
    vector_norm = torch.sqrt(torch.sum(points * points, 2, keepdim=False))
    
    if pca:
        # Using PCA to define 3 axises
        _, _, V = torch.pca_lowrank(points)
        axis1, axis2, axis3 = V.chunk(3, dim=2)
        axis1 = axis1.squeeze()
        axis2 = axis2.squeeze()
        axis3 = axis3.squeeze()
        axis1 = axis1 / (torch.norm(axis1, 2, 1, keepdim=True) + 1e-7)
        axis2 = axis2 / (torch.norm(axis2, 2, 1, keepdim=True) + 1e-7)
        axis3 = axis3 / (torch.norm(axis3, 2, 1, keepdim=True) + 1e-7)
        
    elif axis1 is not None:
        axis1, axis2, axis3 = axis1, axis2, axis3
        
    c1 = torch.sum(points * axis1.unsqueeze(1), 2, keepdim=True)
    c2 = torch.sum(points * axis2.unsqueeze(1), 2, keepdim=True)
    c3 = torch.sum(points * axis3.unsqueeze(1), 2, keepdim=True)
        
    new_c = torch.cat([c1, c2, c3], 2)
    assert (new_c.size() == torch.Size([batch_size, num_point, 3]))
    return axis1, axis2, axis3, new_c


def inverse_point_ortho_feature(axis1, axis2, axis3, points):
    # Axis shape: [batch_size, 3]
    # Input shape: [batch_size, num_point, 3]
    # Return: [batch_size, num_point, 3]
    if len(list(points.size())) == 2 and len(list(axis1.size())) == 1:
        axis1 = axis1[None,:]
        axis2 = axis2[None,:]
        axis3 = axis3[None,:]
        points = points[None,:]

    batch_size = points.size()[0]
    num_point = points.size()[1]
    
    A = torch.cat([axis1.unsqueeze(2), axis2.unsqueeze(2), axis3.unsqueeze(2)], dim=2)
    A = torch.transpose(A, 1, 2).unsqueeze(1).repeat(1, num_point, 1, 1)
    X = torch.matmul(torch.linalg.pinv(A), points.unsqueeze(3)).squeeze()
    
    return X.squeeze()

In [18]:
a, b, c = V.chunk(3, dim=2)
print(a[0])
print(b[0])
print(c[0])

tensor([[-0.9997],
        [ 0.0233],
        [-0.0085]])
tensor([[ 0.0087],
        [ 0.0097],
        [-0.9999]])
tensor([[0.0232],
        [0.9997],
        [0.0099]])


In [17]:
input_clone = torch.clone(input_data)

In [97]:
rotation_transform(dataset_test)

In [18]:
X = inverse_point_projection_feature(axis1, axis2, axis3, dataset_new)

In [19]:
device = torch.device('cuda:0')
data_old = input_clone.to(device)
data_rec = X.to(device)
cd_p, cd_t = calc_cd(data_old, data_rec)
print(cd_p.mean())
print(cd_t.mean())

tensor(2.1600e-06, device='cuda:0')
tensor(4.2582e-07, device='cuda:0')


In [20]:
log_dir = 'images/temp'
idx_to_plot = [i for i in range(0, 1600, 75)]
save_path = log_dir
os.makedirs(save_path, exist_ok=True)
for idx in idx_to_plot:
    pic = 'object_%d.png' % idx
    #ply = 'object_%d.ply' % idx
    plot_single_pcd(X[idx], os.path.join(log_dir, pic))
    #pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(X[idx]))
    #o3d.io.write_point_cloud(os.path.join(log_dir, ply), pcd)

In [101]:
data_new = torch.cat([dataset_new, torch.zeros(400, 2048, 1)], 2)
print(data_new.size())
data_rot = torch.cat([n_dataset_new, torch.zeros(400, 2048, 1)], 2)
print(data_rot.size())

torch.Size([400, 2048, 5])
torch.Size([400, 2048, 5])


In [48]:
z = 100000
dummy = np.vstack((np.zeros(z), np.zeros(z), np.ones(z))).T
print(dummy.shape)
for i in range(z):
    azi = np.random.rand() * 2 * np.pi
    R = np.array(((np.cos(azi), np.sin(azi), 0),
                              (-np.sin(azi), np.cos(azi), 0),
                              (0, 0, 1)))
    x = np.random.rand(2)
    v = np.array((np.cos(2*np.pi*x[0])*np.sqrt(x[1]),
                              np.sin(2*np.pi*x[0])*np.sqrt(x[1]),
                              np.sqrt(1-x[1])))
    H = np.eye(3) - 2 * np.outer(v, v)
    rotation_matrix = -H @ R
    dummy[i] = dummy[i] @ rotation_matrix
plot_single_pcd(dummy, os.path.join('images/temp', 'uniform.png'))
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(dummy))
o3d.io.write_point_cloud('images/temp/uniform.ply', pcd)

dummy = np.vstack((np.zeros(z), np.zeros(z), np.ones(z))).T
for i in range(z):
    angs = np.random.rand(3) * 2 * np.pi
    rot_z = np.array(((np.cos(angs[0]), -np.sin(angs[0]), 0),
                                  (np.sin(angs[0]), np.cos(angs[0]), 0),
                                  (0, 0, 1)))
    rot_y = np.array(((np.cos(angs[1]), 0, np.sin(angs[1])),
                                  (0, 1, 0),
                                  (-np.sin(angs[1]), 0, np.cos(angs[1]))))
    rot_x = np.array(((1, 0, 0),
                                  (0, np.cos(angs[2]), -np.sin(angs[2])),
                                  (0, np.sin(angs[2]), np.cos(angs[2]))))
    rotation_matrix = rot_z @ rot_y @ rot_x
    dummy[i] = dummy[i] @ rotation_matrix
plot_single_pcd(dummy, os.path.join('images/temp', 'euler.png'))
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(dummy))
o3d.io.write_point_cloud('images/temp/euler.ply', pcd)

(100000, 3)


True

In [5]:
model_module = importlib.import_module('.%s' % args.model_name, 'models')
net = torch.nn.DataParallel(model_module.Model(args))
net.cuda()
net.module.load_state_dict(torch.load(args.load_model)['net_state_dict'])
logging.info("%s's previous weights loaded." % args.model_name)

Loaded compiled 3D CUDA chamfer distance
INFO:root:siamese_vrcnet's previous weights loaded.


In [16]:
# azimuthal_angle = 90
# angle coordinates are askewed
#rotation_matrix = np.array(((0,0,1),(0,1,0),(-1,0,0)))
#rotation_matrix = np.array(((0,-1,0),(1,0,0),(0,0,1)))
dataset_plot = dataset_test.gt_data.copy()
#dataset_plot = dataset_test.gt_data @ rotation_matrix
print(dataset_plot.shape)

(1600, 2048, 3)


In [11]:
import numpy as np
#np.random.seed(5)
for i in range(dataset_test.gt_data.shape[0]):
    angs = np.random.rand(3) * 2 * np.pi
    rot_z = np.array(((np.cos(angs[0]), -np.sin(angs[0]), 0),
                      (np.sin(angs[0]), np.cos(angs[0]), 0),
                      (0, 0, 1)))
    rot_y = np.array(((np.cos(angs[1]), 0, np.sin(angs[1])),
                      (0, 1, 0),
                      (-np.sin(angs[1]), 0, np.cos(angs[1]))))
    rot_x = np.array(((1, 0, 0),
                      (0, np.cos(angs[2]), -np.sin(angs[2])),
                      (0, np.sin(angs[2]), np.cos(angs[2]))))
    rotation_matrix = rot_z @ rot_y @ rot_x
    #rotation_matrix = np.array(((0,0,1),(0,1,0),(-1,0,0)))
    #dataset_test.gt_data[i] = dataset_test.gt_data[i] @ rotation_matrix
    dataset_test.input_data[26*i:26*i+25] = dataset_test.input_data[26*i:26*i+25] @ rotation_matrix
print(dataset_test.gt_data.shape)
print(dataset_test.input_data.shape)

(1600, 2048, 3)
(41600, 2048, 3)


In [5]:
import numpy as np
for i in range(dataset_test.gt_data.shape[0]):
    ang = np.random.rand() * 2 * np.pi
    rotation_matrix = np.array(((np.cos(ang), 0, np.sin(ang)),
                      (0, 1, 0),
                      (-np.sin(ang), 0, np.cos(ang))))
    dataset_test.gt_data[i] = dataset_test.gt_data[i] @ rotation_matrix
    dataset_test.input_data[26*i:26*i+25] = dataset_test.input_data[26*i:26*i+25] @ rotation_matrix
print(dataset_test.gt_data.shape)
print(dataset_test.input_data.shape)

(1600, 2048, 3)
(41600, 2048, 3)


In [12]:
metrics = ['cd_p', 'cd_t', 'f1']
test_loss_meters = {m: AverageValueMeter() for m in metrics}
test_loss_cat = torch.zeros([16, 3], dtype=torch.float32).cuda()
cat_num = torch.ones([16, 1], dtype=torch.float32).cuda() * 150
cat_name = ['airplane', 'cabinet', 'car', 'chair', 'lamp', 'sofa', 'table', 'vessel',
            'bed', 'bench', 'bookshelf', 'bus', 'guitar', 'motorbike', 'pistol', 'skateboard']
idx_to_plot = [i for i in range(0, 41600, 75)]
logging.info('Testing...')
if args.save_vis:
    save_gt_path = os.path.join(log_dir, 'pics', 'gt')
    save_partial_path = os.path.join(log_dir, 'pics', 'partial')
    save_completion_path = os.path.join(log_dir, 'pics', 'completion')
    os.makedirs(save_gt_path, exist_ok=True)
    os.makedirs(save_partial_path, exist_ok=True)
    os.makedirs(save_completion_path, exist_ok=True)

INFO:root:Testing...


In [ ]:
with torch.no_grad():
    for i, data in enumerate(dataloader_test):
            
        label, inputs_cpu, gt_cpu = data
        # mean_feature = None

        inputs = inputs_cpu.float().cuda()
        gt = gt_cpu.float().cuda()
        inputs = inputs.transpose(2, 1).contiguous()
        # result_dict = net(inputs, gt, is_training=False, mean_feature=mean_feature)
        result_dict = net(inputs, gt, is_training=False)
        for k, v in test_loss_meters.items():
            v.update(result_dict[k].mean().item())

        for j, l in enumerate(label):
            for ind, m in enumerate(metrics):
                test_loss_cat[int(l), ind] = result_dict[m][int(j)]

        if i % args.step_interval_to_print == 0:
            logging.info('test [%d/%d]' % (i, dataset_length / args.batch_size))

        if args.save_vis:
            for j in range(args.batch_size):
                idx = i * args.batch_size + j
                if idx in idx_to_plot:
                    pic = 'object_%d.png' % idx
                    plot_single_pcd(result_dict['out2'][j].cpu().numpy(), os.path.join(save_completion_path, pic))
                    plot_single_pcd(gt_cpu[j], os.path.join(save_gt_path, pic))
                    plot_single_pcd(inputs_cpu[j].cpu().numpy(), os.path.join(save_partial_path, pic))

INFO:root:test [0/1300]


In [13]:
print(np.unique(dataset_test.labels))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


In [ ]:
with torch.no_grad():
    logging.info('Loss per category:')
    category_log = ''
    for i in range(16):
        category_log += '\ncategory name: %s ' % (cat_name[i])
        for ind, m in enumerate(metrics):
            scale_factor = 1 if m == 'f1' else 10000
            category_log += '%s: %f ' % (m, test_loss_cat[i, 0] / cat_num[i] * scale_factor)
    logging.info(category_log)

    logging.info('Overview results:')
    overview_log = ''
    for metric, meter in test_loss_meters.items():
        overview_log += '%s: %f ' % (metric, meter.avg)
    logging.info(overview_log)